In [1]:
from cassandra.cluster import Cluster
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import  Model
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine import connection
from cassandra.query import dict_factory
from datetime import datetime, timedelta
import time
import cassandra
import random
import uuid
import math
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import datetime
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
host = 'localhost'
port = '3306'
db_name = 'Data_Warehouse'
user = 'root'
password = '1'
url = 'jdbc:mysql://' + host + ':' + port + '/' + db_name
driver = "com.mysql.cj.jdbc.Driver"

keyspace = 'study_data_engineering'
cassandra_login = 'cassandra'
cassandra_password = 'cassandra'
cluster = Cluster()
session = cluster.connect(keyspace)

In [3]:

def get_data_from_job(user,password,host,database):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=database)
    query = """select id as job_id,campaign_id , group_id , company_id from job"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data

def get_data_from_publisher(user,password,host,database):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=database)
    query = """select distinct(id) as publisher_id from master_publisher"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data


def generating_dummy_data(n_records,session,user,password,host,db_name):
    publisher = get_data_from_publisher(user,password,host,db_name)
    publisher = publisher['publisher_id'].to_list()
    jobs_data = get_data_from_job(user,password,host,db_name)
    job_list = jobs_data['job_id'].to_list()
    campaign_list = jobs_data['campaign_id'].to_list()
    company_list = jobs_data['company_id'].to_list()
    group_list = jobs_data[jobs_data['group_id'].notnull()]['group_id'].astype(int).to_list()
    i = 0 
    fake_records = n_records
    while i <= fake_records:
        create_time = str(cassandra.util.uuid_from_time(datetime.datetime.now()))
        bid = random.randint(0,1)
        interact = ['click','conversion','qualified','unqualified']
        custom_track = random.choices(interact,weights=(70,10,10,10))[0]
        job_id = random.choice(job_list)
        publisher_id = random.choice(publisher)
        group_id = random.choice(group_list)
        campaign_id = random.choice(campaign_list)
        ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        sql = """ INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('{}',{},{},'{}',{},{},{},'{}')""".format(create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts)
        print(sql)
        session.execute(sql)
        i+=1 
    return print("Data Generated Successfully")


In [4]:
status = "ON"
while status == "ON":
    generating_dummy_data(n_records = random.randint(1,20),session = session , user = user , password = password , host = host , db_name = db_name )
    time.sleep(20)


C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('df8004fe-2be4-11ef-9796-9976a5bb1015',1,12,'click',17,878,1,'2024-06-16 13:32:27')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('df82b6e0-2be4-11ef-9524-344ec5e7969b',1,48,'click',10,1788,27,'2024-06-16 13:32:27')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('df84b81e-2be4-11ef-a272-9ddd85e40b26',0,1,'click',41,978,38,'2024-06-16 13:32:27')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('df877dce-2be4-11ef-8a94-6ce9638e4608',1,10,'click',21,1180,38,'2024-06-16 13:32:27')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('df897408-2be4-11ef-9e56-091a8716fece',1,61,'click',32,1300,31,'2024-06-16 13:32:27')
 INSERT INTO tracking (create_time,bid,campaign_i

C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('ec1c73fa-2be4-11ef-8d7c-6214819bb999',0,1,'qualified',15,1539,20,'2024-06-16 13:32:48')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('ec1f1fc4-2be4-11ef-b673-20d3664a7251',0,48,'unqualified',10,137,28,'2024-06-16 13:32:48')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('ec2134da-2be4-11ef-9a26-47e5f5c674a7',1,144,'click',41,2020,32,'2024-06-16 13:32:48')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('ec238118-2be4-11ef-8105-4fa7bf50806b',1,90,'unqualified',18,2044,30,'2024-06-16 13:32:48')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('ec25b9e2-2be4-11ef-84d7-7474de723de1',1,70,'click',27,2020,5,'2024-06-16 13:32:48')
 INSERT INTO tracking (create_t

C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('f8b75634-2be4-11ef-adc8-1cde6438c7c2',0,10,'click',32,1555,20,'2024-06-16 13:33:09')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('f8ba0082-2be4-11ef-a13f-6b9b237b36eb',0,1,'click',10,502,9,'2024-06-16 13:33:09')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('f8bc6b74-2be4-11ef-9c91-725515525a86',1,193,'conversion',26,102,32,'2024-06-16 13:33:09')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('f8bebf5a-2be4-11ef-b753-c5f618e0dc5c',1,11,'click',18,1416,11,'2024-06-16 13:33:09')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('f8c14e8c-2be4-11ef-bf3a-5adae38506bd',1,61,'click',16,1122,34,'2024-06-16 13:33:09')
 INSERT INTO tracking (create_time,bid,camp

C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_7208\1639888444.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('05362e3a-2be5-11ef-94c1-aafd231c368e',1,193,'conversion',10,1001,35,'2024-06-16 13:33:30')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('053891ac-2be5-11ef-aa1f-d61ac7031290',0,12,'click',17,180,17,'2024-06-16 13:33:30')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('053acc9c-2be5-11ef-bc15-7934de0d704e',0,63,'click',12,1973,10,'2024-06-16 13:33:30')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('053d44f4-2be5-11ef-b713-c8659908f4af',1,70,'click',41,1397,22,'2024-06-16 13:33:30')
 INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('053faa5a-2be5-11ef-bfcb-7eef938139d0',1,11,'click',32,587,15,'2024-06-16 13:33:30')
 INSERT INTO tracking (create_time,bid,ca

KeyboardInterrupt: 